![Clothing Classifier Model](Clothing%20Classifier%20Model.png)


Fashion Forward is a new AI-based e-commerce clothing retailer.
They want to use image classification to automatically categorize new product listings, making it easier for customers to find what they're looking for. It will also assist in inventory management by quickly sorting items.

As a data scientist tasked with implementing a garment classifier, your primary objective is to develop a machine learning model capable of accurately categorizing images of clothing items into distinct garment types such as shirts, trousers, shoes, etc.


In [8]:
# Run the cells below first

In [9]:
!pip install torchmetrics

Defaulting to user installation because normal site-packages is not writeable


In [10]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchmetrics import Accuracy, Precision, Recall

In [11]:
# Load datasets
from torchvision import datasets
import torchvision.transforms as transforms

train_data = datasets.FashionMNIST(root='./data', train=True, download=True, transform=transforms.ToTensor())
test_data = datasets.FashionMNIST(root='./data', train=False, download=True, transform=transforms.ToTensor())

In [12]:
# Start coding here
# Use as many cells as you need

In [13]:
class CNNClassifier(nn.Module):
    def __init__(self, num_classes=10):
        super(CNNClassifier, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.fc = nn.Linear(32 * 14 * 14, num_classes)
    
    def forward(self, x):
        x = self.features(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)
        return x

# Initialize the model
model = CNNClassifier(num_classes=10)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Data loaders
train_loader = DataLoader(train_data, batch_size=64, shuffle=True)
test_loader = DataLoader(test_data, batch_size=64, shuffle=False)

# Training loop
for epoch in range(1):  # Run for 1 or 2 epochs
    model.train()
    for images, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

In [14]:
# Evaluation
model.eval()
accuracy_metric = Accuracy(task="multiclass", num_classes=10)
precision_metric = Precision(task="multiclass", num_classes=10, average=None)
recall_metric = Recall(task="multiclass", num_classes=10, average=None)
predicted = []  # Corrected list name to store predictions

with torch.no_grad():
    for images, labels in test_loader:
        outputs = model(images)
        preds = outputs.argmax(dim=1)
        predicted.extend(preds.tolist())  # Store predictions in the correctly named list
        accuracy_metric.update(preds, labels)
        precision_metric.update(preds, labels)
        recall_metric.update(preds, labels)

# Calculate metrics
accuracy = accuracy_metric.compute()
precision = precision_metric.compute()
recall = recall_metric.compute()

# Print metrics
print("Accuracy:", accuracy.item())  # .item() to get Python number from tensor
print("Precision per class:", precision.numpy())  # Convert tensor to numpy array for easy viewing
print("Recall per class:", recall.numpy())

Accuracy: 0.8777999877929688
Precision per class: [0.85499465 0.9739479  0.8833333  0.8201247  0.7319502  0.9766734
 0.69868994 0.9434914  0.9805924  0.9379243 ]
Recall per class: [0.796 0.972 0.742 0.921 0.882 0.963 0.64  0.935 0.96  0.967]
